In [93]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
df=pd.read_csv("./train.csv")
y=df['Survived']
X=df.drop(['Survived'],axis=1)
X.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [94]:
passenger_ID=X['PassengerId']
X=X.drop(['Ticket','Cabin','Name','PassengerId'],axis=1)

In [95]:
X=pd.get_dummies(X,columns=['Sex','Embarked','Pclass'],prefix=['Sex','Embarked','Pclass'])

In [96]:
X=X.fillna({'Age':29.7})

In [97]:
bestfeatures = SelectKBest(score_func=chi2, k="all")
fit = bestfeatures.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))

         Specs        Score
3         Fare  4518.319091
4   Sex_female   170.348127
5     Sex_male    92.702447
9     Pclass_1    55.175151
11    Pclass_3    41.553071
0          Age    24.689547
6   Embarked_C    20.464401
2        Parch    10.097499
10    Pclass_2     6.160767
8   Embarked_S     5.984840


In [98]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [99]:
clf=LogisticRegression(random_state=0).fit(X_train,y_train)
y_predict=clf.predict(X_test)
accuracy_score(y_test,y_predict)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8026905829596412

In [100]:
df_test=pd.read_csv('./test.csv')
passenger_ID_kaggle=df_test['PassengerId']
X_test_kaggle=df_test.drop(['PassengerId','Name','Ticket','Cabin'],axis=1)

In [101]:
X_test_kaggle=X_test_kaggle.fillna({'Age':X_test_kaggle['Age'].mean(),'Fare':X_test_kaggle['Fare'].mean()})

In [102]:
X_test_kaggle=pd.get_dummies(X_test_kaggle,columns=['Sex','Embarked','Pclass'],prefix=['Sex','Embarked','Pclass'])

In [103]:
X_test_kaggle=scaler.fit_transform(X_test_kaggle)

In [104]:
y_predict_kaggle=clf.predict(X_test_kaggle)

In [106]:
df_submit=pd.DataFrame(list(zip(passenger_ID_kaggle, y_predict_kaggle)),columns=['PassengerId','Survived'])

In [108]:
df_submit.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [109]:
df_submit.to_csv("submit2.csv",index=False)